In [21]:
import pandas as pd
movies=pd.read_csv('/movies.csv')
ratings=pd.read_csv('/ratings.csv')
user_movie_df = pd.pivot_table(ratings, index='userId', columns='movieId', values='rating')


user_movie_df.fillna(0, inplace=True)

In [22]:
user_movie_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
movies=pd.read_csv('/movies.csv')
ratings=pd.read_csv('/ratings.csv')
movies["year"]=movies["title"].str.extract(r'\((\d{4})\)')
merged_df = pd.merge(movies, ratings, on='movieId')

In [24]:
def assign_category(row):
    try:
        if int(row['year']) < 2000:
            return 'old'
        else:
            return 'new'
    except:
        return 'unknown'

merged_df['category'] = merged_df.apply(assign_category, axis=1)

result = merged_df.groupby(['userId', 'category'])['rating'].mean().reset_index()
result

,userId,category,rating
0,1,new,4.600000
1,1,old,4.361233
2,2,new,3.961538
3,2,old,3.833333
4,3,new,0.500000
...,...,...,...
1064,608,new,3.689286
1065,608,old,2.852087
1066,609,old,3.270270
1067,610,new,3.562192


In [25]:
def assign_binary(row):
    if row['category'] == 'old':
        return [int(row['rating'] > 4), 0]
    else:
        return [0, int(row['rating'] > 4)]

result[['old', 'new']] = result.apply(assign_binary, axis=1, result_type='expand')
new_result = result.groupby('userId')[['old', 'new']].max().reset_index()
new_result

,userId,old,new
0,1,1,1
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0
...,...,...,...
605,606,0,0
606,607,0,0
607,608,0,0
608,609,0,0


In [26]:
joined_movies = pd.merge(user_movie_df, new_result, on='userId')

In [27]:
joined_movies

,userId,1,2,3,4,5,6,7,8,9,...,193571,193573,193579,193581,193583,193585,193587,193609,old,new
0,1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
606,607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
607,608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
608,609,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [28]:
merged_df

,movieId,title,genres,year,userId,rating,timestamp,category
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0,964982703,old
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0,847434962,old
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5,1106635946,old
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5,1510577970,old
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5,1305696483,old
...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017,184,4.0,1537109082,new
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,184,3.5,1537109545,new
100833,193585,Flint (2017),Drama,2017,184,3.5,1537109805,new
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018,184,3.5,1537110021,new


In [29]:
new_merged = merged_df.set_index(['userId', 'rating'])['genres'].str.split('|', expand=True).stack().reset_index(name='genres').drop('level_2', axis=1)
new_merged = new_merged.groupby(['userId', 'genres'])['rating'].mean().reset_index()

new_merged['rating'] = (new_merged['rating'] > 4).astype(int)

result = new_merged.pivot(index='userId', columns='genres', values='rating').reset_index().fillna(0)
result=result.drop(columns=["(no genres listed)"])
result

genres,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
607,608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
joined_movies_genres = pd.merge(joined_movies, result, on='userId')
joined_movies_genres

,userId,1,2,3,4,5,6,7,8,9,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
607,608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,609,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
merged_df['day_of_the_week'] = pd.to_datetime(merged_df['timestamp'], unit='s').dt.dayofweek

In [32]:
result = merged_df.groupby(['userId', 'day_of_the_week']).size().reset_index(name='count')
result

,userId,day_of_the_week,count
0,1,1,1
1,1,6,231
2,2,5,29
3,3,4,39
4,4,0,86
...,...,...,...
1377,610,0,3
1378,610,2,866
1379,610,3,10
1380,610,5,403


In [33]:
max_counts = result.groupby('userId')['count'].transform('max')

# filter the dataframe to only include rows with the maximum count for each user
filtered_df = result[result['count'] == max_counts]
filtered_df

,userId,day_of_the_week,count
1,1,6,231
2,2,5,29
3,3,4,39
4,4,0,86
9,5,4,44
...,...,...,...
1359,606,1,397
1367,607,5,130
1370,608,1,376
1376,609,1,37


In [34]:
new_df = pd.DataFrame({'userId': filtered_df['userId']})

# add columns for each day of the week and fill them with 0s
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in days_of_week:
    new_df[day] = 0

new_df

,userId,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,1,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1359,606,0,0,0,0,0,0,0
1367,607,0,0,0,0,0,0,0
1370,608,0,0,0,0,0,0,0
1376,609,0,0,0,0,0,0,0


In [35]:
for _, row in filtered_df.iterrows():
    new_df_row = new_df[new_df['userId'] == row['userId']]
    
    new_df.at[new_df_row.index[0], days_of_week[row['day_of_the_week']]] = 1
new_df

,userId,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,1,0,0,0,0,0,0,1
2,2,0,0,0,0,0,1,0
3,3,0,0,0,0,1,0,0
4,4,1,0,0,0,0,0,0
9,5,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
1359,606,0,1,0,0,0,0,0
1367,607,0,0,0,0,0,1,0
1370,608,0,1,0,0,0,0,0
1376,609,0,1,0,0,0,0,0


In [36]:
joined_days_of_week=pd.merge(joined_movies_genres, new_df, on='userId')

In [37]:
joined_days_of_week

,userId,1,2,3,4,5,6,7,8,9,...,Thriller,War,Western,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0,0,0,0,0,0,1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0,0,0,0,0,1,0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0,0,0,0,1,0,0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,0
4,5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
607,607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0,0,0,0,0,1,0
608,608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
609,609,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0


In [38]:
user_movie_df = pd.pivot_table(ratings, index='userId', columns='movieId', values='rating')

user_movie_df.fillna(-1, inplace=True)

def discretize_review(review):
    if review==-1:
        return "none"
    elif review<3:
        return 'low'
    elif review == 3:
        return 'medium'
    else:
        return 'high'

user_movie_df["userId"]=user_movie_df.index
df_melt = pd.melt(user_movie_df, id_vars='userId', var_name='movie_id', value_name='review')

# Discretize the review values
df_melt['review'] = df_melt['review'].apply(discretize_review)
df_melt["movie_review"]=df_melt["movie_id"].astype(str)+" "+df_melt["review"]
df_melt.drop('review',axis=1,inplace=True)
df_melt.drop('movie_id',axis=1,inplace=True)
df_melt = df_melt[~df_melt['movie_review'].str.contains('none')]

# Pivot the dataframe
df_pivot = df_melt.pivot_table(index='userId', columns='movie_review', aggfunc=len, fill_value=0)

df_pivot

movie_review,1 high,1 low,1 medium,10 high,10 low,10 medium,100 high,100 low,100 medium,100044 high,...,99846 medium,99853 high,999 high,999 low,999 medium,99910 high,99910 medium,99917 high,99917 medium,99992 medium
userId,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
cols_to_delete=[]
for col in joined_days_of_week.columns:
    if isinstance(col,int):
        cols_to_delete.append(col)
joined_days_of_week=joined_days_of_week.drop(columns=cols_to_delete)
joined_days_of_week

,userId,old,new,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Thriller,War,Western,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0,0,0,0,0,0,1
1,2,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0,0,0,0,0,1,0
2,3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0,0,0,0,1,0,0
3,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,0
4,5,0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,606,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
607,607,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0,0,0,0,0,1,0
608,608,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
609,609,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0


In [41]:
final_df = pd.merge(joined_days_of_week, df_pivot, on='userId')

In [46]:
final_df=final_df.drop(columns=['userId'])

In [49]:
final_df = final_df.astype(int)

In [50]:
final_df

,old,new,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,99846 medium,99853 high,999 high,999 low,999 medium,99910 high,99910 medium,99917 high,99917 medium,99992 medium
0,1,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(final_df, min_support=0.1, use_colnames=True)

association_rules(frequent_itemsets, 
                  metric='confidence', 
                  min_threshold=0.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(new),(old),0.209493,0.260229,0.124386,0.593750,2.281643,0.069870,1.820974
1,(old),(Action),0.260229,0.194763,0.139116,0.534591,2.744834,0.088433,1.730172
2,(Action),(old),0.194763,0.260229,0.139116,0.714286,2.744834,0.088433,2.589198
3,(Adventure),(old),0.235679,0.260229,0.158756,0.673611,2.588531,0.097426,2.266532
4,(old),(Adventure),0.260229,0.235679,0.158756,0.610063,2.588531,0.097426,1.960113
...,...,...,...,...,...,...,...,...,...
12701,"(2571 high, 5952 high)","(4993 high, 7153 high, 1196 high, 260 high)",0.186579,0.126023,0.104746,0.561404,4.454773,0.081233,1.992668
12702,"(4993 high, 260 high)","(2571 high, 7153 high, 1196 high, 5952 high)",0.162029,0.114566,0.104746,0.646465,5.642713,0.086183,2.504513
12703,"(7153 high, 260 high)","(2571 high, 1196 high, 4993 high, 5952 high)",0.163666,0.111293,0.104746,0.640000,5.750588,0.086531,2.468631
12704,"(2571 high, 4993 high)","(7153 high, 1196 high, 5952 high, 260 high)",0.204583,0.124386,0.104746,0.512000,4.116211,0.079299,1.794290
